In [ ]:
import numpy as np
import pandas as pd

from bikewheelcalc import BicycleWheel, Rim, Hub, ModeMatrix
from bikewheelcalc.continuum_analysis import calc_continuum_stiff
from doetools import LateralStiffnessDOE, TensionBucklingDOE

from shutil import copy2
from imp import find_module

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

%matplotlib inline

pd.set_option('precision', 2)

## Wheel parameters for "fat bike"

In [ ]:
w_r = 0.080
h_r = 0.005
t_r = 0.001

w = BicycleWheel()
w.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
w.rim = Rim.box(radius=0.3,
                young_mod=69.0e9, shear_mod=26.0e9,
                w=w_r, h=h_r, t=t_r)

w.lace_radial(n_spokes=36, diameter=2.0e-3, young_mod=200e9, offset=0.)

print('A:   {:5.0f} [mm^2]'.format(w.rim.area * 1e6))
print('EI1  {:5.0f} [N m^2]'.format(w.rim.young_mod*w.rim.I33))
print('EI2  {:5.0f} [N m^2]'.format(w.rim.young_mod*w.rim.I22))
print('GJ   {:5.0f} [N m^2]'.format(w.rim.shear_mod*w.rim.I11))
print('EA_s {:5.0f} [kN]'.format(w.spokes[0].EA / 1000))

## Tension Buckling DOE

In [ ]:
Tc_dir = '../data/abaqus/spk_offset_Tc'

# Create ABAQUS simulations
up = 0.2e-3
opts = {'spk_paired': False, 'spk_eltype': 'beam', 'sim_temp_factor': 2.0,
        'rim_profile': 'box', 'rim_width': w_r, 'rim_height': h_r, 'rim_thick': t_r,
        'rim_perturb': [0., up, up, up, up]}

doe = TensionBucklingDOE(out_dir=Tc_dir, opts=opts)

spk_offset = np.arange(-0.020, 0.0201, 0.005)
for o in spk_offset:

    w.lace_radial(n_spokes=36, diameter=2.0e-3, young_mod=200e9, offset=o)
    w.apply_tension(0.)

    jobname = 'Tc_offset_{:.3f}'.format(o)
    doe.add_experiment(w, opts={'jobname': jobname, 'spk_offset': o, 'spk_T': 0.})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=1)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_tension.py',
      dst=Tc_dir);

## Lateral stiffness DOE

In [ ]:
klat_dir = '../data/abaqus/spk_offset_Klat'

# Create ABAQUS simulations
opts = {'spk_paired': False, 'spk_eltype': 'beam',
        'rim_profile': 'box', 'rim_width': w_r,
        'rim_height': h_r, 'rim_thick': t_r}

doe = LateralStiffnessDOE(out_dir=klat_dir, opts=opts)

spk_offset = np.arange(-0.024, 0.024001, 0.002)
for o in spk_offset:

    w.lace_radial(n_spokes=36, diameter=2.0e-3, young_mod=200e9, offset=o)
    w.apply_tension(0.)

    jobname = 'Klat_offset_{:.3f}'.format(o)
    doe.add_experiment(w, opts={'jobname': jobname, 'spk_offset': o, 'spk_T': 0.})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=2)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_lat_stiffness.py',
      dst=klat_dir);

In [ ]:
# Load DOE database
print('Loading database...')
doe_k = LateralStiffnessDOE(out_dir=klat_dir, db_file=klat_dir+'/_doe_db.csv')

# Populate DOE database with results
print('Extracting results...')
d = doe_k.extract_results()
print('\nDone')

def rr_lateral_stiffness(r, smeared_spokes=True, buckling=True, coupling=True):
    w = doe_k.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.1)

    rr = ModeMatrix(w, N=36)
    
    return rr.calc_lat_stiff(smeared_spokes=smeared_spokes,
                             buckling=buckling,
                             coupling=coupling)

doe_k.db['K_lat_cp_sm'] = doe_k.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=True), axis=1)
doe_k.db['K_lat_cp_ds'] = doe_k.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=False, coupling=True), axis=1)
doe_k.db['K_lat_nc_sm'] = doe_k.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=False), axis=1)

# doe_k.db['Tc_theor'] = doe_k.db.apply(lambda x: bl.calc_buckling_tension(doe.wheel_from_row(x))[0], axis=1)

In [ ]:
def rr_buckling_tension(r):
    w = doe_t.wheel_from_row(r)
    
    rr = ModeMatrix(w, N=36)
    print(w.spokes[0].rim_pt)
    return rr.buckling_tension(smeared_spokes=False, coupling=True)

def rr_Tc_max(r):
    ld = pd.read_csv(doe_t.out_dir + '/' + r.name + '_Pd.csv')
    return ld['Tension [N]'].max()

# Load DOE database
print ('Loading database...')
doe_t = TensionBucklingDOE(out_dir=Tc_dir, db_file=Tc_dir+'/_doe_db.csv')

# Populate DOE database with results
print ('Extracting results...')
d = doe_t.extract_results()
doe_t.db['Tc_rr'] = doe_t.db.apply(lambda x: rr_buckling_tension(x), axis=1)
doe_t.db['Tc_max'] = doe_t.db.apply(lambda x: rr_Tc_max(x), axis=1)

print ('\nDone')

In [ ]:
fig, ax = plt.subplots()

ax.plot(doe_t.db['spk_offset'], doe_t.db['Tc'], 'k')
ax.plot(doe_t.db['spk_offset'], doe_t.db['Tc_rr'], 'ro')
ax.plot(doe_t.db['spk_offset'], doe_t.db['Tc_max'], 'b*')

ax.set_ylim([0., 16000])

In [ ]:
klat_T_dir = '../data/abaqus/spk_offset_Klat_tension'

# Create ABAQUS simulations
opts = {'spk_paired': False, 'spk_eltype': 'beam',
        'rim_profile': 'box', 'rim_width': w_r,
        'rim_height': h_r, 'rim_thick': t_r}

doe = LateralStiffnessDOE(out_dir=klat_T_dir, opts=opts)

spk_offset = np.arange(-0.024, 0.024001, 0.002)
for o in [-0.01, 0., 0.01]:
    
    w.lace_radial(n_spokes=36, diameter=2.0e-3, young_mod=200e9, offset=o)
    
    Tc = doe_t.db.loc['Tc_offset_{:.3f}'.format(o), 'Tc_max']
    for Tn in np.linspace(0., 0.9, 19):
        
        w.apply_tension(Tn*Tc)

        jobname = 'Klat_offset_{:.3f}_Tn_{:.2f}'.format(o, Tn)
        doe.add_experiment(w, opts={'jobname': jobname, 'spk_offset': o, 'spk_T': Tn*Tc, 'spk_Tn': Tn})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=4)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_lat_stiffness.py',
      dst=klat_T_dir);

In [ ]:
# Load DOE database
print('Loading database...')
doe_kT = LateralStiffnessDOE(out_dir=klat_T_dir, db_file=klat_T_dir+'/_doe_db.csv')

# Populate DOE database with results
print('Extracting results...')
d = doe_kT.extract_results()
print('\nDone')

def rr_lateral_stiffness(r, smeared_spokes=True, buckling=True, coupling=True):
    w = doe_kT.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.1)

    rr = ModeMatrix(w, N=36)
    
    return rr.calc_lat_stiff(smeared_spokes=smeared_spokes,
                             buckling=buckling,
                             coupling=coupling)

doe_kT.db['K_lat_cp_sm'] = doe_kT.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=True), axis=1)
doe_kT.db['K_lat_cp_ds'] = doe_kT.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=False, coupling=True), axis=1)
doe_kT.db['K_lat_nc_sm'] = doe_kT.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=False), axis=1)

In [ ]:
with plt.style.context(('seaborn-paper')):

    fig, ax = plt.subplots(ncols=2, figsize=(4.25, 2.25),sharey=True)

    ax[0].plot([  0.,   0.], [0., 200.], '--', color='#999999')
    ax[0].plot([-10., -10.], [0., 200.], '--', color='#999999')
    ax[0].plot([ 10.,  10.], [0., 200.], '--', color='#999999')
    
    ax[0].plot(doe_k.db['spk_offset']*1000, doe_k.db['K_lat_nc_sm']/1000, 'k', label='Eqn ...')
    ax[0].plot(doe_k.db['spk_offset']*1000, doe_k.db['K_lat_cp_ds']/1000, 'C1-.', label='Eqn ...')
    ax[0].plot(doe_k.db['spk_offset']*1000, doe_k.db['K_lat_abq']/1000, 'C0*', label='ABAQUS')

    
    ax[0].set_xlabel('Spoke offset [mm]')
    ax[0].set_ylabel('Lateral stiffness [N/mm]')
    
    ax[0].set_ylim([0., 200.])
    ax[0].set_xlim([-25., 25.])
    
    ax[0].legend(loc='best')
    
    
    for i, g in doe_kT.db.groupby('spk_offset'):
        ax[1].plot(g['spk_T']/1000, g['K_lat_abq'] / 1000, 'C0*')
        ax[1].plot(g['spk_T']/1000, g['K_lat_cp_ds'] / 1000, 'C1-.')
        ax[1].plot(g['spk_T']/1000, g['K_lat_nc_sm'] / 1000, 'k-')
        
    ax[1].set_xlabel('Spoke tension [kN]')
    
    ax[1].set_xlim([0., 8.])
    ax[1].set_xticks(range(0, 9, 2))

    plt.tight_layout()
    plt.savefig('../figs/stress_analysis/_python_spoke_offset.pdf')

In [ ]:
def y_cr_spk(dd, r):
    wh = dd.wheel_from_row(r)
    
    kbar = calc_continuum_stiff(wh, tension=0.)
    
    w, v = np.linalg.eig(kbar[np.ix_([0, 3], [0, 3])])
    
    eig0 = v[:, np.argmin(w)]
    return eig0[0] / eig0[1]

mu = (w.rim.shear_mod*w.rim.I11) / (w.rim.young_mod*w.rim.I22)

y_cr = lambda n: (1 + mu*n**2) / ((1 + mu)*n**2)
print(y_cr(2), y_cr(3))

doe_k.db['y_cr_spk'] = doe_k.db.apply(lambda x: y_cr_spk(doe_k, x), axis=1)